In [9]:
import sqlite3
import pandas as pd 
import numpy as np
import pickle 

# XGBoost
import xgboost as xgb
from sklearn.model_selection import train_test_split 

## SQL query

In [2]:
# Connect to the database
conn = sqlite3.connect('./database/old_car_db.db')

# Create a cursor
c = conn.cursor()

In [3]:
# Take the train table 
query = open('./database/script.sql', 'r').read()
df = pd.read_sql_query(query, conn)
df.head()

,Name,Location,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,Price,Brand,...,Mitsubishi,Nissan,Porsche,Renault,Skoda,Smart,Tata,Toyota,Volkswagen,Volvo
0,Maruti Wagon R LXI CNG,Mumbai,72000,3,21.01,998.0,58.16,5.0,1.75,Maruti,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Hyundai Creta 1.6 CRDi SX Option,Pune,41000,3,19.67,1582.0,126.20,5.0,12.50,Hyundai,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Honda Jazz V,Chennai,46000,3,18.20,1199.0,88.70,5.0,4.50,Honda,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Maruti Ertiga VDI,Chennai,87000,3,20.77,1248.0,88.76,7.0,6.00,Maruti,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,40670,2,15.20,1968.0,140.80,5.0,17.74,Audi,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Model training

In [6]:
X,y = df.drop(['Name', 'Location', 'Brand','Price'],axis=1),df["Price"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [7]:
# {'learning_rate': 0.3, 'max_depth': 3, 'n_estimators': 300}
model = xgb.XGBRegressor(learning_rate=0.3, max_depth=3, n_estimators=300)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.3, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=300, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [8]:
# Evaluate the model
model.score(X_test, y_test)

0.9165335000171995

In [11]:
# Save the model
pickle.dump(model, open('sql_model.pkl', 'wb'))